# Capstone Project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

We will explore to find an optimal location for a restaurant in this project. Specifically, this report will be targeted to **opening an Chinese restaurant in Austin, Texas, America**.

We will **try to detect locations that are with least number of Chinese restaurant**, We would also prefer **locations as close to The University of Texas at Austin possible**.

We want to help people who are interested in investing a Chinese restaurant in Austin and bring more delicious foods to this fascinating city.

## Data <a name="data"></a>

We will mostly utilise **FourSquare data API and Google Maps Api**. Venue information are obtained via FourSquare API based on coordiates and approximate addresses of University of Austin obtained from Google Maps API

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 8x8 killometers centered around The University of Texas at Austin.

Let's first find the latitude & longitude of The University of Texas at Austin, using Google Maps geocoding API.

In [1]:
api_key = ''
google_api_key = ''

In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'The University of Texas at Austin, Austin, Texas, America'
UT_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, UT_center))

Coordinate of The University of Texas at Austin, Austin, Texas, America: [30.2849185, -97.7340567]


Now let's create a grid of area candidates, equaly spaced, centered around The University of Texas at Austin and within ~4km. Our neighborhoods will be defined as circular areas with a radius of 200 meters, so our neighborhood centers will be 400 meters apart.

In [3]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('UT center longitude={}, latitude={}'.format(UT_center[1], UT_center[0]))
x, y = lonlat_to_xy(UT_center[1], UT_center[0])
print('UT center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('UT center longitude={}, latitude={}'.format(lo, la))

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
Coordinate transformation check
-------------------------------
UT center longitude=-97.7340567, latitude=30.2849185
UT center UTM X=-6444599.621400241, Y=13716454.099847103
UT center longitude=-97.73405670000326, latitude=30.284918499998057


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
UT_center_x, UT_center_y = lonlat_to_xy(UT_center[1], UT_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells（4KM为半径）
x_min = UT_center_x - 4000
x_step = 400
y_min = UT_center_y - 4000 - (int(21/k)*k*400 - 8000)/2
y_step = 400 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 200 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(UT_center_x, UT_center_y, x, y)
        if (distance_from_center <= 4001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [5]:
!pip install folium

import folium

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!pip install --upgrade pip

     |████████████████████████████████| 1.4MB 3.0MB/s 
  Found existing installation: pip 19.2.2
    Uninstalling pip-19.2.2:
      Successfully uninstalled pip-19.2.2


In [7]:
map_UT = folium.Map(location=UT_center, zoom_start=13)
folium.Marker(UT_center, popup='dobie').add_to(map_UT)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT)
    folium.Circle([lat, lon], radius=200, color='blue', fill=False).add_to(map_UT)
map_UT

In [11]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, UT_center[0], UT_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(UT_center[0], UT_center[1], addr))

Reverse geocoding check
-----------------------
Address of [30.2849185, -97.7340567] is: 2139 San Jacinto Blvd, Austin, TX 78712, USA


In [12]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Austin', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [12]:
addresses[150:170]

['705 W 32nd St, TX 78705, USA',
 '3200 Grandview St, TX 78705, USA',
 '1104 W 34th St, TX 78705, USA',
 '2002 S L Davis Ave, TX 78702, USA',
 '1901 New York Ave, TX 78702, USA',
 '1204 Salina St, TX 78702, USA',
 '1407 Leona St, TX 78702, USA',
 '1600 Comal St, TX 78702, USA',
 '1300 E Martin Luther King Jr Blvd, TX 78722, USA',
 'Printing and Press Building, 2100 Comal St, TX 78722, USA',
 'Mike A. Myers Track and Soccer Stadium, 707 Clyde Littlefield Dr, TX 78705, USA',
 'University Police Building, 2201 Robert Dedman Dr, TX 78705, USA',
 'E. William Doty Fine Arts Building, 2301 Trinity St, TX 78712, USA',
 'Service Building, 304 E 24th St, TX 78712, USA',
 '204 E Dean Keeton St, TX 78712, USA',
 '101 E 27th St, TX 78712, USA',
 '210 W 27th St, TX 78705, USA',
 '306 W 29th St, TX 78705, USA',
 '2927 Guadalupe St, TX 78705, USA',
 '3005 Washington Square, TX 78705, USA']

Looking good. Let's now place all this into a Pandas dataframe.

In [13]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y
0,"1712 E 40th St, TX 78722, USA",3994.996871,30.293084,-97.710743,-6.445800e+06,1.371264e+07
1,"4008 Cherrywood Rd, TX 78722, USA",3893.584467,30.294771,-97.712342,-6.445400e+06,1.371264e+07
2,"1401 Wilshire Blvd, TX 78722, USA",3831.448812,30.296458,-97.713942,-6.445000e+06,1.371264e+07
3,"4104 Wildwood Rd, TX 78722, USA",3810.511777,30.298145,-97.715541,-6.444600e+06,1.371264e+07
4,"1000 E 41st St, TX 78751, USA",3831.448812,30.299832,-97.717140,-6.444200e+06,1.371264e+07
5,"1017 E 43rd St, TX 78751, USA",3893.584467,30.301520,-97.718740,-6.443800e+06,1.371264e+07
6,"4309 Red River St, TX 78751, USA",3994.996871,30.303207,-97.720340,-6.443400e+06,1.371264e+07
7,"1801 E 38th 1/2 St, TX 78722, USA",3903.844259,30.289352,-97.710028,-6.446400e+06,1.371299e+07
8,"3808 Sycamore Dr, TX 78722, USA",3736.308338,30.291039,-97.711627,-6.446000e+06,1.371299e+07
9,"3816 Cherrywood Rd, TX 78722, USA",3605.551275,30.292726,-97.713226,-6.445600e+06,1.371299e+07


...and let's now save/persist this data into local file.

In [14]:
df_locations.to_pickle('./locations.pkl')

### Foursquare
let's use Foursquare API to get info on restaurants in each neighborhood.

In [15]:
# The code was removed by Watson Studio for sharing.
foursquare_client_id = '' # your Foursquare ID
client_id =''
CLIENT_SECRET = '' # your Foursquare Secret
foursquare_client_secret = ''
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + foursquare_client_id)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GAQ4ITLPIALMBNNYFTJXITPXOMXTJPHV3DMYIC0CJIDYOSEW
CLIENT_SECRET:KNGVV40XND1ZG30245N0JPDMC2V31TLEFFOMR5DZYADTMBFS


In [16]:
# Category IDs corresponding to Chinese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

chinese_restaurant_categories = ['4bf58dd8d48988d145941735','52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3',
'4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef','52af3aaa3cf9994f4e043bf0','52af3ac83cf9994f4e043bf3','52af3afc3cf9994f4e043bf8',
'52af3b463cf9994f4e043bfe','52af3b593cf9994f4e043c00','52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04','52af3b913cf9994f4e043c06',
'52af0bd33cf9994f4e043bdd']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', US', '')
    address = address.replace(', America', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [17]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found Chinese restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    chinese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter=chinese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_chinese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_chinese:
                    chinese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, chinese_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
chinese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('chinese_restaurants_350.pkl', 'rb') as f:
        chinese_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, chinese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('chinese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(chinese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [18]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Chinese restaurants:', len(chinese_restaurants))
print('Percentage of Chinese restaurants: {:.2f}%'.format(len(chinese_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 386
Total number of Chinese restaurants: 12
Percentage of Chinese restaurants: 3.11%
Average number of restaurants in neighborhood: 2.3351648351648353


In [19]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('506909e0e4b08e8fd4aeb63e', 'Whole Foods Lamar Bistro', 30.270947067595255, -97.75355882944697, 'Austin, TX 78701, United States', 201, False, -6444602.369253948, 13720472.61224629)
('4b9c4cd7f964a5205a5d36e3', 'El Naranjo', 30.259492947925935, -97.73828979453252, '85 Rainey St, Austin, TX 78701, United States', 137, False, -6447764.943044489, 13719934.539123548)
('52856dab11d222ec617db017', 'BBQ Revolution', 30.31522995854258, -97.71649996225851, '2421 Webberville Rd, Austin, TX 78702, United States', 313, False, -6442087.293415852, 13710774.555807255)
('5612ee42498ec1a21c3392f3', 'Bistro Vonish', 30.315239460044662, -97.71655142990497, '701 E 53rd St (53rd Street), Austin, TX, United States', 309, False, -6442080.759142875, 13710779.764259847)
('4c0c1823009a0f476122ecbf', 'Golden Chick', 30.272292179011618, -97.69798645832043, '1144 E. Airport Blvd., Austin, TX 78702, United States', 218, False, -6450029.134915865, 13713496.739391709)


In [18]:
print('List of Chinese restaurants')
print('---------------------------')
for r in list(chinese_restaurants.values())[:15]:
    print(r)
print('...')
print('Total:', len(chinese_restaurants))

List of Chinese restaurants
---------------------------
('5b187cc4d48ec1002c9c89d7', 'Lin Asian Bar + Dim Sum Restaurant', 30.27286216354746, -97.75797745504693, '1203 W 6th St, Austin, TX 78703, United States', 217, True, -6443885.062147885, 13720791.171667444)
('4a465a7cf964a520baa81fe3', 'Magic Wok', 30.293054810398377, -97.74168408535371, '2716 Guadalupe St, Austin, TX 78705, United States', 330, True, -6442679.456677801, 13716442.358391128)
('5a808e91b9a5a8633feee10a', 'China Family', 30.282663, -97.742545, '1914 Guadalupe St, Austin, TX 78705, United States', 272, True, -6444060.678142866, 13717757.655062266)
('43e823a7f964a5200e2f1fe3', "P.F. Chang's", 30.26378172414975, -97.74141311645508, '201 San Jacinto Blvd (at E 2nd St), Austin, TX 78701, United States', 315, True, -6446842.96621801, 13719817.978980178)
('49d927d9f964a5200b5e1fe3', 'Chinatown Downtown', 30.26728994636488, -97.74396733061214, '109 W 5th St (Colorado St.), Austin, TX 78701, United States', 293, True, -644608

In [20]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: Patrizi's, Dai Due Butcher Shop & Supper Club
Restaurants around location 103: Patrizi's, Dai Due Butcher Shop & Supper Club, Mi Madre's Restaurant, Abo Youssef
Restaurants around location 104: Star Seeds Cafe
Restaurants around location 105: Star Seeds Cafe, Yoshi Ramen Austin, Austin's Habibi
Restaurants around location 106: Yoshi Ramen Austin, Austin's Habibi
Restaurants around location 107: 
Restaurants around location 108: 
Restaurants around location 109: 
Restaurants around location 110: 


Let's now see all the collected restaurants in our area of interest on map, and let's also show Chinese restaurants in different color.

In [21]:
map_UT = folium.Map(location=UT_center, zoom_start=13)
folium.Marker(UT_center, popup='Alexanderplatz').add_to(map_UT)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_chinese = res[6]
    color = 'red' if is_chinese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_UT)
map_UT

So now we have all the restaurants in area within few kilometers from the University of Texas at Austin, and we know which ones are Chinese restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of the University of Texas at Austin that have low restaurant density, particularly those with low number of Chinese restaurants. We will limit our analysis to area ~6km around city center.

In first step: we have collected the required **data: location and type (category) of every restaurant within 6km from the University of Texas at Austin**. We have also **identified Chinese restaurants** (according to Foursquare categorization).

Second step: in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Austin - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Chinese restaurants in vicinity) and focus our attention on those areas.

In third and final step: we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Chinese restaurants in radius of 1000 meters**. Moreover, we will use data from Foursquare to explore the venues in these promising areas to check the feasibility. Finally, We will present map of all such locations but also create clusters of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [22]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 2.3351648351648353


,Address,Distance from center,Latitude,Longitude,X,Y,Restaurants in area
0,"1712 E 40th St, TX 78722, USA",3994.996871,30.293084,-97.710743,-6.445800e+06,1.371264e+07,0
1,"4008 Cherrywood Rd, TX 78722, USA",3893.584467,30.294771,-97.712342,-6.445400e+06,1.371264e+07,1
2,"1401 Wilshire Blvd, TX 78722, USA",3831.448812,30.296458,-97.713942,-6.445000e+06,1.371264e+07,0
3,"4104 Wildwood Rd, TX 78722, USA",3810.511777,30.298145,-97.715541,-6.444600e+06,1.371264e+07,10
4,"1000 E 41st St, TX 78751, USA",3831.448812,30.299832,-97.717140,-6.444200e+06,1.371264e+07,10
5,"1017 E 43rd St, TX 78751, USA",3893.584467,30.301520,-97.718740,-6.443800e+06,1.371264e+07,0
6,"4309 Red River St, TX 78751, USA",3994.996871,30.303207,-97.720340,-6.443400e+06,1.371264e+07,2
7,"1801 E 38th 1/2 St, TX 78722, USA",3903.844259,30.289352,-97.710028,-6.446400e+06,1.371299e+07,0
8,"3808 Sycamore Dr, TX 78722, USA",3736.308338,30.291039,-97.711627,-6.446000e+06,1.371299e+07,0
9,"3816 Cherrywood Rd, TX 78722, USA",3605.551275,30.292726,-97.713226,-6.445600e+06,1.371299e+07,4


OK, now let's calculate the **distance to nearest Chinese restaurant from every area candidate center** 

In [23]:
distances_to_chinese_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in chinese_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_chinese_restaurant.append(min_distance)

df_locations['Distance to Chinese restaurant'] = distances_to_chinese_restaurant

In [24]:
df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y,Restaurants in area,Distance to Chinese restaurant
0,"1712 E 40th St, TX 78722, USA",3994.996871,30.293084,-97.710743,-6.445800e+06,1.371264e+07,0,1148.641757
1,"4008 Cherrywood Rd, TX 78722, USA",3893.584467,30.294771,-97.712342,-6.445400e+06,1.371264e+07,1,806.671898
2,"1401 Wilshire Blvd, TX 78722, USA",3831.448812,30.296458,-97.713942,-6.445000e+06,1.371264e+07,0,549.600960
3,"4104 Wildwood Rd, TX 78722, USA",3810.511777,30.298145,-97.715541,-6.444600e+06,1.371264e+07,10,522.879412
4,"1000 E 41st St, TX 78751, USA",3831.448812,30.299832,-97.717140,-6.444200e+06,1.371264e+07,10,751.494873
5,"1017 E 43rd St, TX 78751, USA",3893.584467,30.301520,-97.718740,-6.443800e+06,1.371264e+07,0,1084.475085
6,"4309 Red River St, TX 78751, USA",3994.996871,30.303207,-97.720340,-6.443400e+06,1.371264e+07,2,1451.698272
7,"1801 E 38th 1/2 St, TX 78722, USA",3903.844259,30.289352,-97.710028,-6.446400e+06,1.371299e+07,0,1642.688161
8,"3808 Sycamore Dr, TX 78722, USA",3736.308338,30.291039,-97.711627,-6.446000e+06,1.371299e+07,0,1244.896003
9,"3816 Cherrywood Rd, TX 78722, USA",3605.551275,30.292726,-97.713226,-6.445600e+06,1.371299e+07,4,849.180619


In [25]:
print('Average distance to closest Chinese restaurant from each area center:', df_locations['Distance to Chinese restaurant'].mean())

Average distance to closest Chinese restaurant from each area center: 1334.2595176956854


In [26]:
UT_boroughs_url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/TX/Austin.geo.json'

OK, so **on average Chinese restaurant can be found within ~1350m** from every area center candidate. We have to filter specific locations.

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Austin boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km from the University of Texas Austin.

In [27]:
UT_boroughs = requests.get(UT_boroughs_url).json()

def boroughs_style(feature):
    return { 'color': 'blue', 'fill': False }

In [28]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

chinese_latlons = [[res[2], res[3]] for res in chinese_restaurants.values()]

In [29]:
from folium import plugins
from folium.plugins import HeatMap

map_UT = folium.Map(location=UT_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_UT) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_UT)
folium.Marker(UT_center).add_to(map_UT)
folium.Circle(UT_center, radius=1000, fill=False, color='white').add_to(map_UT)
folium.Circle(UT_center, radius=2000, fill=False, color='white').add_to(map_UT)
folium.Circle(UT_center, radius=3000, fill=False, color='white').add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Looks like a few pockets of low restaurant density closest to city center can be found **north, north-east and east from the University of Texas at Austin**. 

Let's create another heatmap map showing **heatmap/density of Chinese restaurants** only.

In [30]:
map_UT = folium.Map(location=UT_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_UT) #cartodbpositron cartodbdark_matter
HeatMap(chinese_latlons).add_to(map_UT)
folium.Marker(UT_center).add_to(map_UT)
folium.Circle(UT_center, radius=1000, fill=False, color='white').add_to(map_UT)
folium.Circle(UT_center, radius=2000, fill=False, color='white').add_to(map_UT)
folium.Circle(UT_center, radius=3000, fill=False, color='white').add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

This map is definitely not  'hot' (Chinese restaurants represent a subset of ~3% of all restaurants near the University of Texas at Austin) but it also indicates higher density of existing Chinese restaurants south and west from the University of Texas at Austin , with closest pockets of **low Chinese restaurant density positioned east, south-east and south from city center**.

Based on this we will now focus our analysis on areas *north and east from the University of Texas at Austin * - we will move the center of our area of interest and reduce it's size to have a radius of **2.5km**. This places our location candidates in boroughs **Hyde Park and Central East Austin**

### Hyde Park and Central East Austin
These booming area are close to university, many students choose to live there, boroughs Hyde Park and Central East Austin appear to justify further analysis.

In [31]:
roi_x_min = UT_center_x - 2500
roi_y_max = UT_center_y + 500
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_UT = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_UT)
folium.Marker(UT_center).add_to(map_UT)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.4).add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Not bad - this nicely covers all the pockets of low restaurant density in Hyde Park and Central East Austin closest to UT center.

Let's also create new, more dense grid of location candidates restricted to our new region of interest(200m appart)

In [32]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 200
y_step = 200 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

565 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Chinese restaurant**.

In [33]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_chinese_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, chinese_restaurants)
    roi_chinese_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [34]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Chinese restaurant':roi_chinese_distances})

df_roi_locations.head(10)

,Distance to Chinese restaurant,Latitude,Longitude,Restaurants nearby,X,Y
0,1191.407916,30.300327,-97.711989,0,-6.444650e+06,1.371195e+07
1,925.770097,30.296140,-97.709433,0,-6.445500e+06,1.371213e+07
2,979.385850,30.296984,-97.710233,0,-6.445300e+06,1.371213e+07
3,1066.947620,30.297828,-97.711032,0,-6.445100e+06,1.371213e+07
4,1021.786698,30.298671,-97.711832,0,-6.444900e+06,1.371213e+07
5,1014.750654,30.299515,-97.712632,0,-6.444700e+06,1.371213e+07
6,1046.608677,30.300358,-97.713431,0,-6.444500e+06,1.371213e+07
7,1114.028974,30.301202,-97.714231,0,-6.444300e+06,1.371213e+07
8,1211.086862,30.302046,-97.715031,0,-6.444100e+06,1.371213e+07
9,1331.315973,30.302889,-97.715830,0,-6.443900e+06,1.371213e+07


OK. Let us now **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 250 meters**, and **no Chinese restaurants in radius of 1000 meters**.

In [35]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Chinese restaurant']>=1000)
print('Locations with no Chinese restaurants within 1000m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 551
Locations with no Chinese restaurants within 1000m: 432
Locations with both conditions met: 418


Let's see how this looks on a map.

In [36]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_UT = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_UT)
HeatMap(restaurant_latlons).add_to(map_UT)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_UT)
folium.Marker(UT_center).add_to(map_UT)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT) 
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Looking good. We now have a bunch of locations fairly close to the University of Texas at Austin (mostly in Central East Austin and Hyde Park), and we know that each of those locations has no more than two restaurants in radius of 250m, and no Chinese restaurant closer than 1000m. Any of those locations is a potential candidate for a new Chinese restaurant.

Let's now show those good locations in a form of heatmap:

In [37]:
map_UT = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_UT)
folium.Marker(UT_center).add_to(map_UT)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [38]:
from sklearn.cluster import KMeans

number_of_clusters = 12

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_UT = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_UT)
HeatMap(restaurant_latlons).add_to(map_UT)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_UT)
folium.Marker(UT_center).add_to(map_UT)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_UT) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [39]:
map_UT = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(UT_center).add_to(map_UT)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_UT) 
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Let's zoom in on candidate areas in **Hyde Park**:

In [40]:
map_UT = folium.Map(location=[30.30706969999999, -97.7274412], zoom_start=15)
folium.Marker(UT_center).add_to(map_UT)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_UT) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Let's get the venues in **Hyde Park**

In [42]:
neighborhood_latitude = 30.30816969999999
neighborhood_longitude = -97.7274412
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(25)

,name,categories,lat,lng
0,Juiceland,Juice Bar,30.307210,-97.724873
1,The Flag Store of Hyde Park,Beer Store,30.306952,-97.724514
2,Quack's 43rd St Bakery,Bakery,30.304731,-97.726511
3,Antonelli's Cheese Shop,Cheese Shop,30.304378,-97.726498
4,Avenue B Grocery & Market,Sandwich Place,30.309136,-97.731579
5,Shipe Park,Park,30.307105,-97.727837
6,Hyde Park Bar & Grill,American Restaurant,30.304222,-97.726705
7,Antonelli's Cheese House,Cheese Shop,30.304281,-97.726466
8,Elisabet Ney Museum,Art Museum,30.306556,-97.726397
9,Asti Trattoria,Italian Restaurant,30.304794,-97.726176


In [43]:
Hydeparkgrouped = nearby_venues.groupby('categories').count()
Hydeparkgrouped

,name,lat,lng
categories,,,
American Restaurant,1,1,1
Art Museum,1,1,1
Bakery,1,1,1
Bed & Breakfast,1,1,1
Beer Store,1,1,1
Cheese Shop,2,2,2
Convenience Store,1,1,1
Grocery Store,1,1,1
Italian Restaurant,2,2,2


Even no Asia restaurant in  **Hyde Park**, definitely a promising market.

Candidate areas in **Central East Austin**:

In [44]:
map_UT = folium.Map(location=[30.27792349999999, -97.7218647], zoom_start=15)
folium.Marker(UT_center).add_to(map_UT)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_UT) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_UT)
folium.GeoJson(UT_boroughs, style_function=boroughs_style, name='geojson').add_to(map_UT)
map_UT

Let's get the venues in **Central East Austin**

In [46]:
neighborhood_latitude1 = 30.27792349999999
neighborhood_longitude1 = -97.7218647
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(25)

,name,categories,lat,lng
0,Juiceland,Juice Bar,30.307210,-97.724873
1,The Flag Store of Hyde Park,Beer Store,30.306952,-97.724514
2,Quack's 43rd St Bakery,Bakery,30.304731,-97.726511
3,Antonelli's Cheese Shop,Cheese Shop,30.304378,-97.726498
4,Avenue B Grocery & Market,Sandwich Place,30.309136,-97.731579
5,Shipe Park,Park,30.307105,-97.727837
6,Hyde Park Bar & Grill,American Restaurant,30.304222,-97.726705
7,Antonelli's Cheese House,Cheese Shop,30.304281,-97.726466
8,Elisabet Ney Museum,Art Museum,30.306556,-97.726397
9,Asti Trattoria,Italian Restaurant,30.304794,-97.726176


In [47]:
CentralEastAustingrouped = nearby_venues.groupby('categories').count()
CentralEastAustingrouped

,name,lat,lng
categories,,,
American Restaurant,1,1,1
Art Museum,1,1,1
Bakery,1,1,1
Bed & Breakfast,1,1,1
Beer Store,1,1,1
Cheese Shop,2,2,2
Convenience Store,1,1,1
Grocery Store,1,1,1
Italian Restaurant,2,2,2


No Asia restaurant in  **Central East Austin**.

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [49]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', US', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, UT_center_x, UT_center_y)
    print('{}{} => {:.1f}km from UT'.format(addr, ' '*(50-len(addr)), d/1000))

Addresses of centers of areas recommended for further analysis

Eastwoods Neighborhood Park, 3001 Harris Park Ave, Austin, TX 78705A => 1.0km from UT
3902 Brookview Rd, Austin, TX 78722A               => 3.9km from UT
3914 Avenue H, Austin, TX 78751A                   => 3.1km from UT
3011 N Interstate Hwy 35, Austin, TX 78722A        => 1.9km from UT
3505 Greenway St, Austin, TX 78705A                => 2.2km from UT
2102 Comal St, Austin, TX 78722A                   => 1.5km from UT
1300 N Interstate 35 Frontage Rd, Austin, TX 78751A => 4.2km from UT
3304 Grooms St, Austin, TX 78705A                  => 2.2km from UT
Darrell K. Royal TX Memorial Stadium, 2139 San Jacinto Blvd, Austin, TX 78712A => 0.5km from UT
2003 Coleto St, Austin, TX 78722A                  => 2.5km from UT
3207 Walnut Ave, Austin, TX 78722A                 => 3.1km from UT
802 Park Blvd, Austin, TX 78751A                   => 3.6km from UT


This concludes our analysis. We have created 12 addresses representing centers of zones containing locations with low number of restaurants and no Chinese restaurants nearby, all zones being fairly close to the University of Texas at Austin center ( above half of those less than 3km from ). These centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Hyde Park and Central East Austin boroughs.

In [50]:
map_UT = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(UT_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_UT)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_UT) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_UT)
map_UT

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a lot of restaurants near the University of Texas at Austin, there are pockets of low restaurant density in south and east form school. Highest concentration of restaurants was detected south and west from school, so we focused our attention to areas north and east , corresponding to boroughs Hyde Park and Central East Austin. 

Result of all this is 12 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Chinese restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Recommended zones should considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify the University of Austin areas close to center with low number of restaurants (particularly Chinese restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Chinese restaurant. Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like population size，real estate availability, prices, social and economic dynamics of every neighborhood etc.